In [66]:
import numpy as np
import os
import progressbar
import pysam
import pysamstats

from sklearn.model_selection import train_test_split

## Input params

In [62]:
bam_file_path = '/home/diplomski-rad/blade/pb/escherichia-coli-NCTC86/reads-to-ref-sorted.bam'
reference_fasta_path = '/home/data/pacific_biosciences/bacteria/escherichia/coli/escherichia_coli_reference.fasta'
include_indels = True
save_directory_path = './new-class-test'
neighbourhood_size = 3

## Create pileups: X and y together

In [51]:
def _generate_pileups(bam_file_path, reference_fasta_path, include_indels=True):
    """
    Generate pileups from reads alignment to reference.

    Pileups are generated for all contigs.

    :param bam_file_path: path to .bam file containing alignments
    :type bam_file_path: str
    :param reference_fasta_path: path to .fasta file
    :type reference_fasta_path: str
    :param include_indels: flag which indicates whether to include indels in
        pileups
    :type include_indels: bool
    :return: pileups (X)
    :rtype: tuple of np.ndarray and list of str
    """
    bam_file = pysam.AlignmentFile(bam_file_path)

    if include_indels:
        info_of_interest = ['A', 'C', 'G', 'T', 'insertions', 'deletions']
        indel_positions = [4, 5]
    else:
        info_of_interest = ['A', 'C', 'G', 'T']
        
    # Last number in shape - 5 - is for letters other than A, C, G and T.
    mapping = {'A': 0, 'a': 0, 'C': 1, 'c': 1, 'G': 2, 'g': 2, 'T': 3, 't': 3}
    total_options = len(info_of_interest) + 1

    pileups = [np.zeros(
        (bam_file.get_reference_length(contig_name),
         len(info_of_interest)
         )) for contig_name in bam_file.references]
    y_oh = [np.zeros(
        (bam_file.get_reference_length(contig_name),
         total_options
         )) for contig_name in bam_file.references]

    total_length = np.sum(
        [bam_file.get_reference_length(contig_name) for contig_name in
         bam_file.references])
    progress_counter = 0
    with progressbar.ProgressBar(max_value=total_length) as progress_bar:
        for contig_id, contig_name in enumerate(bam_file.references):
            for record in pysamstats.stat_variation(
                    bam_file, chrom=contig_name, fafile=reference_fasta_path):
                progress_bar.update(progress_counter)
                progress_counter += 1
                
                curr_position = record['pos']
                
                # Parsing X.
                # Note: Commented code is slower due to the fact that in python when using list comprehension
                # new list is created everytime which is expensive. In other languages, like C++, you could
                # create one array and use it every time.
#                 curr_pileup = [record[info] for info in info_of_interest]
#                 pileups[contig_id][curr_position] = curr_pileup
                for i, info in enumerate(info_of_interest):
                    pileups[contig_id][curr_position][i] += record[info]
                
                # Parsing y.
#                 argmax_pileup = np.argmax(curr_pileup)
#                 y_oh[contig_id][curr_position][argmax_pileup] = 1
                if not include_indels:
                    y_oh[contig_id][curr_position][mapping.get(record['ref'], -1)] = 1
                else:
                    pileup_argmax = np.argmax(pileups[contig_id][curr_position])
                    if pileup_argmax in indel_positions:
                        y_oh[contig_id][curr_position][pileup_argmax] = 1
                    else:
                        y_oh[contig_id][curr_position][mapping.get(record['ref'], -1)] = 1

    return pileups, y_oh

In [50]:
X, y_oh = _generate_pileups(bam_file_path, reference_fasta_path, include_indels=include_indels)

100% (4641652 of 4641652) |##############| Elapsed Time: 0:00:54 Time:  0:00:54


KeyboardInterrupt: 

## Create pileups: wrapper function

In [52]:
def generate_pileups(bam_file_path, reference_fasta_path, mode,
                     save_directory_path=None, include_indels=True):
    """
    Generates pileups from given alignment stored in bam file.

    Mode must be one of 'training' or 'inference' string indicating pileups
    generation mode. If 'training' is selected, pileups from different
    contigs are all be concatenated. If 'inference' is selected, pileups from
    different contig will be hold separate to enable to make consensus with
    same number of contigs.

    If save_directory_path is provided, generated pileups are stored in that
    directory.

    If include_indels is set to True, indels will also we included in
    pileups. Otherwise, only nucleus bases will be in pileup (A, C, G and T).

    :param bam_file_path: path to .bam file containing alignments
    :type bam_file_path: str
    :param reference_fasta_path: path to .fasta file
    :type reference_fasta_path: str
    :param mode: either 'training' or 'inference' string, representing the
        mode for pileups generation
    :type mode: str
    :param save_directory_path: path to directory for storing pileups
    :type save_directory_path: str
    :param include_indels: flag which indicates whether to include indels in
        pileups
    :type include_indels: bool
    :return: pileups (X) and matching nucleus bases from reference (y) with
        concatenated contigs for inference mode, or separate contigs for
        training mode; also, if save_directory_path is provided, list of
        paths to saved files is returned in tuple (X, y_oh, X_save_paths,
        y_save_paths). In both cases list of contig names is also returned.
    :rtype tuple of np.ndarray and list of str or tuple of np.array and list of str
    """
    _check_mode(mode)

    if save_directory_path is not None:
        if os.path.exists(save_directory_path):
            raise ValueError('You must provide non-existing save output '
                             'directory, {} given.'.format(save_directory_path))
        else:
            os.makedirs(save_directory_path)

    print('##### Generate pileups from read alignments to reference. #####')
    
    X, y_oh = _generate_pileups(bam_file_path,
                                           reference_fasta_path,
                                           include_indels=include_indels)

    total_pileups = len(X)
    if mode == 'training': # training mode
        X, y_oh = [np.concatenate(X, axis=0)], [np.concatenate(y_oh, axis=0)]
        total_pileups = 1
    else:  # inference mode
        pass  # nothing to do

    if save_directory_path is not None:
        X_save_paths = [
            os.path.join(
                save_directory_path,
                'pileups-X-{}{}.npy'.format(
                    i, '-indels' if include_indels else ''))
            for i in range(total_pileups)]
        y_save_paths = [os.path.join(save_directory_path,
                        'pileups-y-{}.npy'.format(i))
                        for i in range(total_pileups)]
        for X_save_path, y_save_path, Xi, yi in zip(
                X_save_paths, y_save_paths, X, y_oh):
            np.save(X_save_path, Xi)
            np.save(y_save_path, yi)
        return X, y_oh, X_save_paths, y_save_paths

    return X, y_oh

def _check_mode(mode):
    """
    Checks if given mode is supported: 'training' or 'inference'.

    :param mode: mode to check
    :type mode: str
    :raise ValueError: if selected mode is not supported
    """
    modes = ['training', 'inference']
    if mode not in modes:
        raise ValueError('You must provide either \'training\' or '
                         '\'inference\' mode, but \'{}\' given.'.format(mode))

In [56]:
X, y_oh, X_save_paths, y_save_paths = generate_pileups(
    bam_file_path,
    reference_fasta_path,
    mode='training', 
    save_directory_path='./new-class-test',
    include_indels=include_indels)

  0% (1022 of 4641652) |                 | Elapsed Time: 0:00:00 ETA:   0:07:34

##### Generate pileups from read alignments to reference. #####


100% (4641652 of 4641652) |##############| Elapsed Time: 0:07:19 Time:  0:07:19


## Create dataset with neighbourhood

In [58]:
def create_dataset_with_neighbourhood(X_paths, y_paths, neighbourhood_size,
                                      mode, save_directory_path=None):
    """
    Creates datasets by mixing all pileups with given neighbourhood_size.

    Datasets are concatenated after extracting neighbourhood_size positions in
    given datasets separately if 'training' mode is selected. If 'inference'
    mode is selected, X_paths are assumed to be paths to different contigs of
    same

    Dataset at i-th position in X_paths should match given labels at i-th
    position in y_paths.

    If save_directory_path is provided, generated pileups are stored in that
    directory.

    :param X_paths: list of paths to X pileup dataset
    :type X_paths: list of str
    :param y_paths: list of paths to y pileup dataset
    :type y_paths: list of str
    :param neighbourhood_size: number of neighbours to use from one size (eg.
        if you set this parameter to 3, it will take 3 neighbours from both
        sides so total number of positions in one example will be 7 -
        counting the middle position)
    :type neighbourhood_size: int
    :param mode: either 'training' or 'inference' string, representing the
        mode for pileups generation
    :type mode: str
    :param save_directory_path: path to directory for storing dataset
    :type save_directory_path: str
    :return:
    :rtype tuple of np.ndarray or tuple of np.array and list of str
    """
    _check_mode(mode)

    if not len(X_paths) == len(y_paths):
        raise ValueError('Number of X_paths and y_paths should be the same!')

    # If training mode is selected, all pileups will be concatenated.
    total_pileups = 1 if mode == 'training' else len(X_paths)

    X_save_paths, y_save_paths = None, None
    if save_directory_path is not None:
        X_save_paths, y_save_paths = _generate_save_paths(neighbourhood_size,
                                                          save_directory_path,
                                                          total_pileups)

    X_neighbourhood_list, y_neighbourhood_list = list(), list()
    for X_path, y_path in zip(X_paths, y_paths):
        print('Parsing ', X_path, ' and ', y_path)

        curr_X, curr_y = np.load(X_path), np.load(y_path)
        # Removing last column which contains everything which was not 'A' nor
        # 'C' nor 'G' nor 'T'.
        curr_y = curr_y[:, :-1]
        new_curr_X, new_curr_y = list(), list()
        empty_rows = _calc_empty_rows(curr_X)

        print('Creating dataset with neighbourhood ...')
        with progressbar.ProgressBar(max_value=curr_X.shape[0]) as progress_bar:
            # TODO(ajuric): Check if this can be speed up.
            for i in range(curr_X.shape[0]):
                progress_bar.update(i)
                if empty_rows[i] == 1:
                    continue  # current row is empty row
                if i < neighbourhood_size or \
                   i >= curr_X.shape[0] - neighbourhood_size:
                    # Current position is not suitable to build an example.
                    continue
                zeros_to_left = np.sum(empty_rows[i - neighbourhood_size:i])
                zeros_to_right = np.sum(
                    empty_rows[i + 1:i + neighbourhood_size + 1])
                if zeros_to_left == 0 and zeros_to_right == 0:
                    new_curr_X.append(
                        curr_X[
                            i - neighbourhood_size:
                            i + neighbourhood_size + 1])
                    new_curr_y.append(curr_y[i])

        X_neighbourhood_list.append(np.array(new_curr_X))
        y_neighbourhood_list.append(np.array(new_curr_y))

    if mode == 'training':
        X_neighbourhood_list = [np.concatenate(X_neighbourhood_list, axis=0)]
        y_neighbourhood_list = [np.concatenate(y_neighbourhood_list, axis=0)]
    else:  # inference mode
        pass  # nothing to do

    if save_directory_path is not None:
        for X_save_path, y_save_path, Xi, yi in zip(
            X_save_paths, y_save_paths, X_neighbourhood_list,
                y_neighbourhood_list):
            np.save(X_save_path, Xi)
            np.save(y_save_path, yi)
        return X_neighbourhood_list, \
               y_neighbourhood_list, \
               X_save_paths, \
               y_save_paths

    return X_neighbourhood_list, y_neighbourhood_list


def _calc_empty_rows(X):
    """
    Calculates which rows in X are empty rows (i.e. all numbers in that row
    are equal to 0).

    :param X: 2-D data
    :type X: np.ndarray
    :return: 1-D array with 1s on positions which correspond to empty rows.
    :rtype: np.ndarray
    """
    empty_row = np.zeros((1, X.shape[1]))  # size is second axis of X
    empty_rows = [int(v) for v in np.all(empty_row == X, axis=1)]
    return empty_rows


def _generate_save_paths(neighbourhood_size, save_directory_path,
                         total_pileups):
    """
    Generates a list of save paths for dataset X and y.

    :param neighbourhood_size: number of neighbours to use from one size (eg.
        if you set this parameter to 3, it will take 3 neighbours from both
        sides so total number of positions in one example will be 7 -
        counting the middle position)
    :type neighbourhood_size: int
    :param save_directory_path: path to directory for storing dataset
    :type save_directory_path: str
    :param total_pileups: total number of pileups to be generated a the end;
        determines the number of save paths to be generated
    :type total_pileups: int
    :return: tuple of list of str
    """
    # Creating save file paths.
    X_save_paths = [os.path.join(
        save_directory_path,
        'X-pileups-n{}-{}.npy'.format(neighbourhood_size, i))
                    for i in range(total_pileups)]
    y_save_paths = [os.path.join(
        save_directory_path,
        'y-pileups-n{}-{}.npy'.format(neighbourhood_size, i))
                    for i in range(total_pileups)]

    for X_save_path, y_save_path in zip(X_save_paths, y_save_paths):
        if os.path.exists(X_save_path) or os.path.exists(y_save_path):
            raise ValueError('Pileups already exists in given save '
                             'directory. Either provide other save '
                             'directory or empty this one.')
    return X_save_paths, y_save_paths


In [63]:
X_neighbourhood_list, y_neighbourhood_list, X_save_paths, y_save_paths = create_dataset_with_neighbourhood(
    X_save_paths, 
    y_save_paths, 
    neighbourhood_size=neighbourhood_size, 
    mode='training', 
    save_directory_path=save_directory_path)

Parsing  ./new-class-test/pileups-X-0-indels.npy  and  ./new-class-test/pileups-y-0.npy


  0% (4240 of 4641652) |                 | Elapsed Time: 0:00:00 ETA:   0:03:38

Creating dataset with neighbourhood ...


100% (4641652 of 4641652) |##############| Elapsed Time: 0:03:37 Time:  0:03:37


## Reshape dataset for conv

In [64]:
def read_dataset_and_reshape_for_conv(X_paths, y_paths, validation_size=None,
                                      save_directory_path=None):
    """
    Reads X and y from given paths and reshapes them for applying in
    convolutional networks.

    Reshaping is done by splitting different letters in separate channels,
    eg. letter 'A' has it's own channel, letter 'C' has it's own channel, etc.

    :param path_X: list of paths to X data
    :type path_X: list of str
    :param path_y: list of paths to y data
    :type path_y: list of str
    :param validation_size: specifies percentage of dataset used for validation
    :type validation_size: float
    :return: If validation_size is None, returns just X and y reshaped. If
    validation_size is float, returns a tuple in following order: (X, y,
    X_train, X_validate, y_train, y_validate).
    :rtype: tuple of np.ndarray
    """
    if not len(X_paths) == len(y_paths):
        raise ValueError('Number of X_paths and y_paths must be the same!')

    if validation_size is not None:
        if validation_size < 0 or validation_size > 1.0:
            raise ValueError('Validation size must be float from [0, 1], but {}'
                             ' given.'.format(validation_size))
        if not len(X_paths) == 1:
            raise ValueError(
                'Validation size can only be provided if there is only one X_path and y_path.')

    X_save_paths, y_save_paths = None, None
    if save_directory_path is not None:
        pass

    X_list, y_list = list(), list()
    for X_path, y_path in zip(X_paths, y_paths):
        X, y = np.load(X_path), np.load(y_path)
        print('X shape before reshaping:', X.shape)
        print('y shape before reshaping:', y.shape)

        new_X = list()
        neighbourhood_size = X[0].shape[0]
        # Number of columns is equal to the number of letters in dataset (A, C,
        # G, T, I, D, ...).
        num_columns = X[0].shape[1]
        num_data = X.shape[0]
        with progressbar.ProgressBar(max_value=num_data) as progress_bar:
            for i, xi in enumerate(X):
                new_xi = np.dstack(
                    [xi[:, col_index].reshape(neighbourhood_size, 1)
                     for col_index in range(num_columns)]
                )
                new_X.append(new_xi)
                progress_bar.update(i)

        new_X = np.array(new_X)
        X = new_X
        print('X shape after reshaping:', X.shape)
        print('y shape after reshaping:', y.shape)

        X_list.append(X), y_list.append(y)

    if validation_size is None:
        return X_list, y_list
    else:
        # There is only one X and y (because, all datasets are concatenated for training).
        X, y = X_list[0], y_list[0]
        print('Splitting to train and validation set.')
        X_train, X_validate, y_train, y_validate = train_test_split(
            X, y, test_size=validation_size)
        print('X_train shape:', X_train.shape)
        print('X_validate shape:', X_validate.shape)
        print('y_train:', y_train.shape)
        print('y_validate:', y_validate.shape)
        return X, y, X_train, X_validate, y_train, y_validate

In [67]:
X, y, X_train, X_validate, y_train, y_validate = read_dataset_and_reshape_for_conv(
    X_save_paths,
    y_save_paths,
    validation_size=0.1)

  0% (2391 of 4495756) |                 | Elapsed Time: 0:00:00 ETA:   0:03:08

X shape before reshaping: (4495756, 7, 6)
y shape before reshaping: (4495756, 6)


100% (4495756 of 4495756) |##############| Elapsed Time: 0:03:01 Time:  0:03:01


X shape after reshaping: (4495756, 7, 1, 6)
y shape after reshaping: (4495756, 6)
Splitting to train and validation set.
X_train shape: (4046180, 7, 1, 6)
X_validate shape: (449576, 7, 1, 6)
y_train: (4046180, 6)
y_validate: (449576, 6)


In [76]:
print(y.shape)
indel_classes = np.sum(y[:, 4:])
ca
print(indel_classes)
print(indel_classes / y.shape[0])

(4495756, 6)
80439.0
0.017892207673192228


In [77]:
print(y.shape)
indel_classes = np.sum(y[:, 0:1])

print(indel_classes)
print(indel_classes / y.shape[0])

(4495756, 6)
1083064.0
0.2409080919871986


In [78]:
print(y.shape)
indel_classes = np.sum(y[:, 1:2])

print(indel_classes)
print(indel_classes / y.shape[0])

(4495756, 6)
1127378.0
0.25076494364907703


In [80]:
print(y.shape)
indel_classes = np.sum(y[:, 2:3])

print(indel_classes)
print(indel_classes / y.shape[0])

(4495756, 6)
1124101.0
0.2500360339840507


In [81]:
print(y.shape)
indel_classes = np.sum(y[:, 3:4])

print(indel_classes)
print(indel_classes / y.shape[0])

(4495756, 6)
1080774.0
0.2403987227064814


In [82]:
print(y.shape)
indel_classes = np.sum(y[:, 4:5])

print(indel_classes)
print(indel_classes / y.shape[0])

(4495756, 6)
1823.0
0.0004054935365709349


In [83]:
print(y.shape)
indel_classes = np.sum(y[:, 5:6])

print(indel_classes)
print(indel_classes / y.shape[0])

(4495756, 6)
78616.0
0.017486714136621295
